In [44]:
# from __future__ import print_function

import sys
import pyspark.sql.types as T
import pytz
import hashlib
import pytz
import re
import hmac
import json
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from datetime import datetime, timedelta
from hashlib import sha256 
from pyspark.sql.functions import udf
from pyspark.sql.functions import lit
from pyspark.sql.column import Column
# 替换手机号
def get_secret_obj():
    df=spark.read.text("s3://rupiahplus-configs/etl/data_secrt/col.json").collect()
    keymap=''
    for  row in df:
        keymap=keymap+row['value']
    json_content=json.loads(keymap)
    return json_content
colmap= get_secret_obj()
mobileType=colmap["mobileType"]
def getRelmobile(colNm,str):
    if colNm in mobileType:
        relmobile = str.strip().replace("+", "").replace("-", "").replace(" ", "").replace('"', '')
        if (relmobile.startswith("62")):
            relmobile = relmobile.replace("62", "0",1)
        if relmobile.startswith("0") == False:
            relmobile = "0" + relmobile;
        return relmobile
    return str
# hmac_256加密
def hmac_sha256(key,colNm,value):
    if value!=None and value!='':
        rtn=getRelmobile(colNm,value)
        h = hmac.new(key.encode('utf-8'),digestmod=sha256)
        h.update(rtn.encode('utf-8'))
        h_str = h.hexdigest()
        return h_str
    return ""
spark.udf.register("hmac_sha256",hmac_sha256,T.StringType())
def getTableColum(b):
    colum=""
    for index in range(len(b)):
        if(index>0 and index<len(b)-8):   
            colum=colum+setDef(b[index]["data_type"],b[index]["col_name"])+", "
    return colum[0:len(colum)-2]
def setDef(type,table_col):
    if(type[:7] == 'decimal'):
        return "ifnull("+table_col+",0) as "+table_col
    else:
        return table_col
def add_secret_col(dbType,tableNm,col):
    secret_Col=[]
    if colmap.get(dbType)!=None and colmap[dbType]!=None and colmap[dbType].get(tableNm)!=None:
        secret_Col=colmap[tablemap[databaseName]][tableNm]
    sh256Col =col
    hmac_key=colmap["hmac_key"]
#     sql="select hmac_sha256('{hmac_key}','123456789') as sh2".format(hmac_key=hmac_key)
    for i in secret_Col:
        sh256Col=sh256Col+ ", hmac_sha256('{hmac_key}','"+i+"', "+i+") "+ i+"_x"
df=spark.read.format("csv").load("s3://rupiahplus-configs/yanzhengmweishiy_副.csv")
# df.createOrReplaceTempView("tempViewName")  
path="s3://rupiahplus-configs/yanzhengmweishiy_副_加密.csv"
# spark.sql("select _c0, hmac_sha256('xUIAZ3grtPOxaPNK','mobile',_c0) mobile1 from  tempViewName").write.mode("overwrite").csv(path)
df.withColumn("mobile",hmac_sha256(lit(hmac_key),lit("mobile"),F.col('_c0'))).show()

# keymap=''
# for  row in df:
#     keymap=keymap+row['value']
# json_content=json.loads(keymap)
# return json_content

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not' when building DataFrame boolean expressions.
Traceback (most recent call last):
  File "<stdin>", line 39, in hmac_sha256
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/column.py", line 690, in __nonzero__
    raise ValueError("Cannot convert column into bool: please use '&' for 'and', '|' for 'or', "
ValueError: Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not' when building DataFrame boolean expressions.

